In [1]:
from sklearn import preprocessing
import numpy as np
import tensorflow as tf

In [2]:
class txtReader:
    def __init__(self, filename):
        self.filename = filename

    def read_split(self):
        with open(self.filename, 'r') as f:
            file_read = f.read()
        efg = []
        lines = file_read.split('\n')
        efg.append(lines)
        text = []
        text_id = []
        for i in lines:
            if i != '':
                word = i.split('\t')
                text.append(word[0])
                text_id.append(word[1])

        return text, text_id

In [7]:
class alphabet:
    def __init__(self, train_file, dev_file, test_file):
        self.train_file = train_file
        self.dev_file = dev_file
        self.test_file = test_file
        self.data = dict()
        self.labels = dict()

    def read_split(self):
        text_files = []
        for text_file in [self.train_file, self.dev_file, self.test_file]:
            txt = txtReader(text_file)
            text, text_id = txt.read_split()
            text_files.append(text)
            text_files.append(text_id)

        return text_files[0], text_files[1], text_files[2], text_files[3], text_files[4], text_files[5]
    
    def _tagger(self, dataset, cnt, dictionary):
        for i in dataset:
            # obtener indice de i en dataset
            pos = dataset.index(i)
            if i not in dictionary:
                dictionary[i] = cnt
                dataset[pos] = cnt
                cnt += 1
            else:
                dataset[pos] = dictionary[i]

        return dataset, cnt, dictionary 

    def labelEncoder(self):
        train, train_id, dev, dev_id, test, test_id = self.read_split()
        cnt = 1
        cnt_id = 1

        train, cnt, self.data = self._tagger(train, cnt, self.data) 
        train_id, cnt_id, self.labels = self._tagger(train_id, cnt_id, self.labels)
        dev, cnt, self.data = self._tagger(dev, cnt, self.data)
        dev_id, cnt_id, self.labels = self._tagger(dev_id, cnt_id, self.labels)
        
        for te in test:
            pos = test.index(te)
            if te not in self.data:
                self.data[te] = -1 # -1 indica que la palabra es desconocida
                test[pos] = -1
            else:
                test[pos] = self.data[te]

        for te_id in test_id:
            pos_id = test_id.index(te_id)
            if te_id not in self.labels:  
                self.labels[te_id] = -1
                test_id[pos_id] = self.labels[te_id]
            else:
                test_id[pos_id] = self.labels[te_id]  

        return train, train_id, dev, dev_id, test, test_id

In [8]:
train, train_id, dev, dev_id, test, test_id = alphabet('materiales_practica/datasets/PartTUT/train.txt', 'materiales_practica/datasets/PartTUT/dev.txt', 'materiales_practica/datasets/PartTUT/test.txt').labelEncoder()

In [8]:
class FFTagger():
    def __init__(self):
        pass

    def model(self, n):
        model = tf.keras.model.Sequential()
        model.add(tf.keras.Input(shape=(n*2+1,)))
        model.add(tf.keras.layers.Embedding(input_dim=1, output_dim=20, input_length=n*2+1))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dense(1, activation='softmax'))
        return model

    def train(self, train, train_id, dev, dev_id, n, model):
        train = np.concatenate((np.zeros(n), train, np.zeros(n)))
        dev = np.concatenate((np.zeros(n), dev, np.zeros(n)))

        # almacenar ventanas de tamaño n*2+1 en una lista de listas para cada conjunto
        train_windows = []
        dev_windows = []

        for i in range(n, len(train) - n):
            # Quedarme con todos los elementos de text que estén entre i-n y i+n
            data = train[i-n:i+n+1]
            train_windows.append(data)
        
        for i in range(n, len(dev) - n):
            data = dev[i-n:i+n+1]
            dev_windows.append(data)

        model.fit(train_windows, train_id, epochs=10, validation_data=(dev_windows, dev_id), verbose=1)

        return model

    # def predict(self, model, test, test_id, n):
    #     test = np.concatenate((np.zeros(n), test, np.zeros(n)))
    #     test_id = np.concatenate((np.zeros(n), test_id, np.zeros(n)))
    #     for i in range(n, len(text) - n):
    #         pred = model.predict(test)


    #     return pred
